In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import torchmetrics

torch.manual_seed(90)
device = torch.device('cpu')


In [11]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])


In [12]:
dataloaders={}

trainset = torchvision.datasets.CIFAR10(
    root='C:/Datasets/CIFAR10', train=True, download=True, transform=transform)
dataloaders['train'] = torch.utils.data.DataLoader(
    trainset, batch_size=64, shuffle=True, num_workers=2)
testset = torchvision.datasets.CIFAR10(
    root='C:/Datasets/CIFAR10', train=False, download=True, transform=transform)
dataloaders['val'] = torch.utils.data.DataLoader(
    testset, batch_size=64, shuffle=True, num_workers=2)


Files already downloaded and verified
Files already downloaded and verified


In [13]:
dataset_sizes={}
dataset_sizes['train']=len(trainset)
dataset_sizes['val']=len(testset)

In [14]:
model=models.resnet18(pretrained=True)

In [15]:
num_ftrs=model.fc.in_features
model.fc=nn.Linear(num_ftrs,10)
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [16]:
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(model.fc.parameters(),lr=0.001,momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)


In [17]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                print("Batch Loss: ",loss.item())
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model


In [18]:
model=train_model(model,criterion,optimizer,exp_lr_scheduler)

Epoch 0/24
----------
Batch Loss:  2.641119956970215
Batch Loss:  2.8595237731933594
Batch Loss:  2.536834239959717
Batch Loss:  2.8082923889160156
Batch Loss:  2.662033796310425
Batch Loss:  2.6233036518096924
Batch Loss:  2.528333902359009
Batch Loss:  2.4920079708099365
Batch Loss:  2.549067735671997
Batch Loss:  2.429476499557495
Batch Loss:  2.554539918899536
Batch Loss:  2.4183270931243896
Batch Loss:  2.3904669284820557
Batch Loss:  2.521681785583496
Batch Loss:  2.531191110610962
Batch Loss:  2.619468927383423
Batch Loss:  2.4155890941619873
Batch Loss:  2.4186294078826904
Batch Loss:  2.501391887664795
Batch Loss:  2.4143545627593994
Batch Loss:  2.4341354370117188
Batch Loss:  2.2094523906707764
Batch Loss:  2.3675670623779297
Batch Loss:  2.322690486907959
Batch Loss:  2.308345079421997
Batch Loss:  2.3605761528015137
Batch Loss:  2.2558748722076416
Batch Loss:  2.3702189922332764
Batch Loss:  2.307368040084839
Batch Loss:  2.2680771350860596
Batch Loss:  2.2456164360046387
